# Harmonize original SSPs to 2023 start date

Assume ssp245 was "correct" history and re-run from there.

In [ ]:
import datetime
import os
import warnings

import aneris.convenience
import numpy as np
import pandas as pd
import scmdata
from dotenv import load_dotenv
from fair import FAIR, __version__
from fair.interface import fill
from fair.io import read_properties
from tqdm.auto import tqdm
import xarray as xr

In [ ]:
# da_emissions_141 = xr.load_dataarray("../data/emissions/v1.4.0/ssp_emissions_1750-2500.nc")
# da_emissions_141

In [ ]:
# emitted_indices = ~np.isnan(da_emissions_141[0, 0, 0, :])
# da_emissions_141['specie'][emitted_indices].to_numpy()

In [ ]:
# hist_ssp245 = da_emissions_141.sel(timepoints=np.arange(1750.5, 2023), scenario='ssp245', config='unspecified', specie=da_emissions_141['specie'][emitted_indices].to_numpy())

In [ ]:
# os.makedirs('../output/')
# hist_ssp245.to_pandas().T.to_csv('../output/historical_ssp245.csv')

In [ ]:
# for scenario in da_emissions_141['scenario'].to_numpy():
#      da_emissions_141.sel(
#          timepoints=np.arange(2022.5, 2500), 
#          scenario=scenario, 
#          config='unspecified', 
#          specie=da_emissions_141['specie'][emitted_indices].to_numpy()
#      ).to_pandas().T.to_csv(f'../output/future_{scenario}.csv')

In [ ]:
df_in = pd.read_csv(
    "../data/emissions/v1.4.0/historical_ssp245.csv"
)

In [ ]:
variables = list(df_in["variable"])
units = list(df_in["unit"])
var_units = {var: unit for var, unit in zip(variables, units)}

times = []
years = range(1750, 2023)
for year in years:
    times.append(datetime.datetime(year, 1, 1))
    # they are really midyears, but we just want this to work

times_future = []
years_future = range(2022, 2500)
for year in years_future:
    times_future.append(datetime.datetime(year, 1, 1))